## 类别不平衡问题
类别不平衡问题，顾名思义，即数据集中存在某一类样本，其数量远多于或远少于其他类样本，从而导致一些机器学习模型失效的问题。例如逻辑回归即不适合处理类别不平衡问题，例如逻辑回归在欺诈检测问题中，因为绝大多数样本都为正常样本，欺诈样本很少，逻辑回归算法会倾向于把大多数样本判定为正常样本，这样能达到很高的准确率，但是达不到很高的召回率。

 类别不平衡问题在很多场景中存在，例如欺诈检测，风控识别，在这些样本中，黑样本（一般为存在问题的样本）的数量一般远少于白样本（正常样本）。

上采样(过采样)和下采样(负采样）策略是解决类别不平衡问题的基本方法之一。上采样即增加少数类样本的数量，下采样即减少多数类样本以获取相对平衡的数据集。

最简单的上采样方法可以直接将少数类样本复制几份后添加到样本集中，最简单的下采样则可以直接只取一定百分比的多数类样本作为训练集。

SMOTE算法是用的比较多的一种上采样算法，其全称是Synthetic Minority Oversampling Technique即合成少数类过采样技术，它是基于随机过采样算法的一种改进方案。SMOTE算法的原理并不是太复杂，用python从头实现也只有几十行代码，但是python的imblearn包提供了更方便的接口，在需要快速实现代码的时候可直接调用imblearn。

imblearn类别不平衡包提供了上采样和下采样策略中的多种接口，基本调用方式一致，主要介绍一下对应的SMOTE方法和下采样中的RandomUnderSampler方法。imblearn可使用pip install imblearn直接安装。
       
## 代码示例
**生成类别不平衡数据**

In [1]:
# 使用sklearn的make_classification生成不平衡数据样本
from sklearn.datasets import make_classification
# 生成一组0和1比例为9比1的样本，X为特征，y为对应的标签
X, y = make_classification(n_classes=2, class_sep=2,
                           weights=[0.9, 0.1], n_informative=3, 
                           n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1, 
                           n_samples=1000, random_state=10)


**查看数据分布**

In [2]:
from collections import Counter
# 查看所生成的样本类别分布，0和1样本比例9比1，属于类别不平衡数据
print(Counter(y))
# Counter({0: 900, 1: 100})


Counter({0: 900, 1: 100})


**SMOTE算法核心语句**

In [4]:
# 使用imlbearn库中上采样方法中的SMOTE接口
from imblearn.over_sampling import SMOTE
# 定义SMOTE模型，random_state相当于随机数种子的作用
smo = SMOTE(random_state=42)
X_smo, y_smo = smo.fit_sample(X, y)

**查看经过SMOTE之后的数据分布**

In [5]:
print(Counter(y_smo))
# Counter({0: 900, 1: 900})

Counter({0: 900, 1: 900})


从上述代码中可以看出，SMOTE模型默认生成一比一的数据，如果想生成其他比例的数据，可以使用radio参数。不仅可以处理二分类问题，同样适用于多分类问题

In [6]:
from imblearn.under_sampling import RandomUnderSampler
# 同理，也可使用ratio来指定下采样的比例
rus = RandomUnderSampler(ratio={0: 500 }, random_state=0)
X_rus, y_rus = rus.fit_sample(X, y)
print(Counter(y_smo))
# Counter({0: 500, 1: 300})


Counter({0: 900, 1: 900})


## [Python实现](https://blog.csdn.net/jiede1/article/details/70215477)

In [7]:
#SMOTE算法及其python实现
import random
from sklearn.neighbors import NearestNeighbors
import numpy as np
class Smote:
    def __init__(self,samples,N=10,k=5):
        self.n_samples,self.n_attrs=samples.shape
        self.N=N
        self.k=k
        self.samples=samples
        self.newindex=0
       # self.synthetic=np.zeros((self.n_samples*N,self.n_attrs))

    def over_sampling(self):
        N=int(self.N/100)
        self.synthetic = np.zeros((self.n_samples * N, self.n_attrs))
        neighbors=NearestNeighbors(n_neighbors=self.k).fit(self.samples)  
        print ('neighbors',neighbors)
        for i in range(len(self.samples)):
            print('samples',self.samples[i])
            nnarray=neighbors.kneighbors(self.samples[i].reshape((1,-1)),return_distance=False)[0]  #Finds the K-neighbors of a point.
            print ('nna',nnarray)
            self._populate(N,i,nnarray)
        return self.synthetic


    # for each minority class sample i ,choose N of the k nearest neighbors and generate N synthetic samples.
    def _populate(self,N,i,nnarray):
        for j in range(N):
            print('j',j)
            nn=random.randint(0,self.k-1)  #包括end
            dif=self.samples[nnarray[nn]]-self.samples[i]
            gap=random.random()
            self.synthetic[self.newindex]=self.samples[i]+gap*dif
            self.newindex+=1
            print(self.newindex)
a=np.array([[1,2,3],[4,5,6],[2,3,1],[2,1,2],[2,3,4],[2,3,4]])
s=Smote(a,N=1000)
s.over_sampling()

neighbors NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)
samples [1 2 3]
nna [0 4 5 3 2]
j 0
1
j 1
2
j 2
3
j 3
4
j 4
5
j 5
6
j 6
7
j 7
8
j 8
9
j 9
10
samples [4 5 6]
nna [1 4 5 0 2]
j 0
11
j 1
12
j 2
13
j 3
14
j 4
15
j 5
16
j 6
17
j 7
18
j 8
19
j 9
20
samples [2 3 1]
nna [2 3 0 4 5]
j 0
21
j 1
22
j 2
23
j 3
24
j 4
25
j 5
26
j 6
27
j 7
28
j 8
29
j 9
30
samples [2 1 2]
nna [3 0 2 4 5]
j 0
31
j 1
32
j 2
33
j 3
34
j 4
35
j 5
36
j 6
37
j 7
38
j 8
39
j 9
40
samples [2 3 4]
nna [4 5 0 3 2]
j 0
41
j 1
42
j 2
43
j 3
44
j 4
45
j 5
46
j 6
47
j 7
48
j 8
49
j 9
50
samples [2 3 4]
nna [4 5 0 3 2]
j 0
51
j 1
52
j 2
53
j 3
54
j 4
55
j 5
56
j 6
57
j 7
58
j 8
59
j 9
60


array([[1.51536415, 2.51536415, 1.96927169],
       [1.2444416 , 2.2444416 , 3.2444416 ],
       [1.80853472, 2.80853472, 3.80853472],
       [1.34279157, 2.34279157, 3.34279157],
       [1.28405372, 1.71594628, 2.71594628],
       [1.70555158, 2.70555158, 3.70555158],
       [1.13646704, 2.13646704, 3.13646704],
       [1.88706365, 2.88706365, 3.88706365],
       [1.        , 2.        , 3.        ],
       [1.        , 2.        , 3.        ],
       [3.3977267 , 4.3977267 , 4.49431676],
       [2.96741036, 3.96741036, 4.96741036],
       [2.40442901, 3.40442901, 4.40442901],
       [2.26162005, 3.26162005, 4.26162005],
       [3.17815527, 4.17815527, 5.17815527],
       [2.86386709, 3.86386709, 4.86386709],
       [3.23401579, 4.23401579, 5.23401579],
       [3.31403795, 4.31403795, 5.31403795],
       [2.20625078, 3.20625078, 1.51562695],
       [3.90268461, 4.90268461, 5.90268461],
       [2.        , 3.        , 3.18793749],
       [2.        , 3.        , 2.12935683],
       [2.

## 参考文章
 1. [类别不平衡问题之SMOTE算法（Python imblearn极简实现）](https://blog.csdn.net/nlpuser/article/details/81265614)
 2. [SMOTE算法(人工合成数据)](https://blog.csdn.net/jiede1/article/details/70215477)